In [1]:
SMAUG128 = {}
SMAUG128['n'] = 256
SMAUG128['p'] = 256
SMAUG128['q'] = 1024
SMAUG128['p_prime'] = 32
SMAUG128['k'] = 2
SMAUG128['h_s'] = 140
SMAUG128['h_r'] = 132
SMAUG128['sigma'] = 1.0625

SMAUG192 = {}
SMAUG192['n'] = 256
SMAUG192['p'] = 256
SMAUG192['q'] = 2048
SMAUG192['p_prime'] = 256
SMAUG192['k'] = 3
SMAUG192['h_s'] = 198
SMAUG192['h_r'] = 151
SMAUG192['sigma'] = 1.453713

SMAUG256 = {}
SMAUG256['n'] = 256
SMAUG256['p'] = 256
SMAUG256['q'] = 2048
SMAUG256['p_prime'] = 64
SMAUG256['k'] = 5
SMAUG256['h_s'] = 176
SMAUG256['h_r'] = 160
SMAUG256['sigma'] = 1.0625

#DGdict = {0: 403163305, 1: 258898250, 2: 68560420, 3: 7487107, 4: 337172, 5: 6262, 6: 48} # sigma = 1.0625
#DGdict = {0: 8993, 1: 7098, 2: 3490, 3: 1069, 4: 204, 5: 24, 6: 2} # sigma = 1.453713

TiGER256 = {}
TiGER256['p'] = 128
TiGER256['q'] = 256
TiGER256['k1'] = 128 
TiGER256['k2'] = 4
TiGER256['h_s'] = 198
TiGER256['h_r'] = 198
TiGER256['h_e'] = 32
TiGER256['n'] = 1024
TiGER256['f'] = 5
TiGER256['thres'] = 256/2

TiGER192 = {}
TiGER192['p'] = 64
TiGER192['q'] = 256
TiGER192['k1'] = 64 
TiGER192['k2'] = 4
TiGER192['h_s'] = 84
TiGER192['h_r'] = 84
TiGER192['h_e'] = 84
TiGER192['n'] = 1024
TiGER192['f'] = 5
TiGER192['thres'] = 256/2

TiGER128 = {}
TiGER128['p'] = 128
TiGER128['q'] = 256 
TiGER128['k1'] = 64 
TiGER128['k2'] = 64 
TiGER128['h_s'] = 160
TiGER128['h_r'] = 128
TiGER128['h_e'] = 32
TiGER128['n'] = 512
TiGER128['f'] = 3
TiGER128['thres'] = 256/2

In [2]:
RR = RealField(5000)
R.<x> = LaurentPolynomialRing(QQ)
sp_t_poly = 1/2*x**-1 + 1/2*x

In [3]:
def unif_error_poly_(p, q):
    
    err_max = int(q/p/2)
    err_list = [i for i in range(err_max+1)]
    poly = 0
    for i in err_list:
        if i == 0 :
            poly += 1
        elif i == err_max :
            poly += 1/2*x**err_max + 1/2*x**(-err_max)
        else :
            poly += x**i + x**(-i)
    
    return poly/(2*err_max)


def e2_poly_(p_prime, q): # Why this way?
    
    err_nums = q/p_prime
    zero_pos = int(err_nums/2-1)
    poly = 0
    for i in range(err_nums):
        if not i == zero_pos:
            poly += x**(i-zero_pos)
        else :
            poly += 1
    
    return poly/err_nums
    
def dist_mult_(poly1, poly2):
    
    poly = 0
    expo_u = vector(poly1.exponents())
    expo_v = vector(poly2.exponents())
    coef_u = vector(poly1.coefficients())
    coef_v = vector(poly2.coefficients())
    
    for i, rows in enumerate(expo_u.tensor_product(expo_v)):
        for j, elts in enumerate(rows):
            poly += coef_u.tensor_product(coef_v)[i][j]*x**elts
    
    return poly

def DGdict_to_DGpoly_(DGdict):
    DGpoly = 0
    for i in DGdict :
        if i == 0 :
            DGpoly += DGdict[i]*x**i
        else :
            DGpoly += DGdict[i]*x**i + DGdict[i]*x**-i
    DGpoly = DGpoly/DGpoly(1)
    
    return DGpoly

In [10]:
def SMAUG_DFP(scheme):
    n = scheme['n']
    p = scheme['p']
    q = scheme['q']
    p_prime = scheme['p_prime']
    h_s = scheme['h_s']
    h_r = scheme['h_r']
    sigma = scheme['sigma']
    
    if not scheme == SMAUG192 :
        DGdict = {0: 403163305, 1: 258898250, 2: 68560420, 3: 7487107, 4: 337172, 5: 6262, 6: 48}
    else :
        DGdict = {0: 8993, 1: 7098, 2: 3490, 3: 1069, 4: 204, 5: 24, 6: 2}
    
    e_poly = DGdict_to_DGpoly_(DGdict)
    e1_poly = unif_error_poly_(p, q)
    #e2_poly = e2_poly_(p_prime, q)
    e2_poly = unif_error_poly_(p_prime, q)
    
    
    #print("e_poly : ", e_poly)
    #print("e1_poly : ", e1_poly)
    #print("e2_poly : ", e2_poly)
    #print("sp_t_poly : ", sp_t_poly)
    
    
    first = (dist_mult_(e_poly, sp_t_poly))**h_r
    second = (dist_mult_(e1_poly, sp_t_poly))**h_s
    last = e2_poly
    
    bit_err_poly = first*second*last
    
    bit_err_prob = 0
    
    for i, coef in enumerate(bit_err_poly.coefficients()):
        if bit_err_poly.exponents()[i] > q/4 :
            bit_err_prob += coef
    
    dfp = 1-(1-bit_err_prob)**n
    
    return dfp.n(100), log(dfp, 2).n(100)

def TiGER_bit_error_prob(scheme):
    
    n = scheme['n']
    p = scheme['p']
    q = scheme['q']
    k1 = scheme['k1']
    k2 = scheme['k2']
    h_r = scheme['h_r']
    h_e = scheme['h_e']
    h_s = scheme['h_s']
    thres = scheme['thres']
    
    uA = unif_error_poly_(p, q)
    
    eB_p = sp_t_poly*(h_e/n)+(1-h_e/n)
    uB_p = unif_error_poly_(k1, q)
    
    
    uB_dp = unif_error_poly_(k2, q)
    eB_dp = sp_t_poly*(h_e/n)+(1-h_e/n)
    
    
    first = (dist_mult_(eB_p*uB_p, sp_t_poly))**h_s
    second = (dist_mult_(uA, sp_t_poly))**h_r
    last = eB_dp*uB_dp
    
    bit_err_poly = first*second*last
    
    bit_err_prob = 0
    
    for i, coef in enumerate(bit_err_poly.coefficients()):
        if bit_err_poly.exponents()[i] > thres :
            bit_err_prob += coef
    
    dfp = RR(1-(1-bit_err_prob)**n)
    
    return dfp.n(100), log(dfp, 2).n(100)

In [9]:
#SMAUG_DFP(SMAUG128)

(2.5485408384989069964663542040e-37, -121.56166803927332631355317713)

In [14]:
SMAUG_DFP(SMAUG192)

(1.7280517119525174196738141478e-42, -138.73183359446854930130545653)

In [15]:
SMAUG_DFP(SMAUG256)

(4.5224905566079258484903241515e-52, -170.56314344377816960584031396)

In [11]:
TiGER_bit_error_prob(TiGER128)

(7.7104221457202531189188323916e-11, -33.594399193884657516200360218)

In [12]:
TiGER_bit_error_prob(TiGER192)

(2.9931287036800042706540284064e-8, -24.993770443454086995633112309)

In [13]:
TiGER_bit_error_prob(TiGER256)

(1.9401167486248758157519388562e-10, -32.263137478113680066544798392)

-102.96190778219411498279474222840787555062435632330467332808338681585440923722067476807574651192142321056902504481279135160070660141499887292891009791213531368459475002254507291992047960855350966544850141167643059654922998955699138582737508596483472229030555504458618540792538950412169329645574210558748959796203707246304200399903125554135981528037733410028454849594333803944431194905958447217878806922446751403074158702257264212109495765488057726572920302112060422134932381660137182904525284327234700955934844376437146386336154154625113515149274376510237575472562726556642676625817983101272598042044785417133897566739470532165749492866637255788157647683164126788352111440689751352134584780292873699138766569089591945984620529487669039600021195722401453180280463031648081742393836202857112522199352956395569876436466817443431045037172387636255017341364909358114681842075041023434195639315888549978482123445953752117017174067214199945905064342572831355290209714292302968501657971307583232521258939927

-143.07302450650843438123658283171546478734765907965433635677799622998364697269374152725544316404006205548561871488955405745539516866196093508710544230559682187197351930155960839339070823594760551077849818176989308530468833367032300429460219591700120446642348233133999906197736407457791021450564509928011151286965068906974356896879924608720046154545325702388289485485738740711303286146356528182506398489577695225300677545098744504264333699059070626772878577048871217372017822682996397582536617052438045140836893008794469735769839641150737395203541303227901512611470585363340612502140852653839223594582733423460517228382851675939387607048846903267606865049473422034260842709974049457652659272440550624816130133527737647700320559363773207074344162594229385730938877213296273629617208106789945117625940906022773929293635667081081184136746764060262611539663669684322263580436000633200093353947945801035751583303268207551314173092414285616749946767842152954013963773875011256814839534117678022272874877482